In [120]:
from string import punctuation
from nltk.corpus import stopwords
from collections import Counter
from scipy import sparse
import numpy as np

### 1. Leitura do texto central

In [121]:
inputFile = "data/BrasCubas-Assis.txt"
with open(inputFile) as f:
    corpus = f.readlines()
    corpus = [paragraph.rstrip() for paragraph in corpus if paragraph != '\n']

### 2. Pré-processamento do texto

In [122]:
punctranslation = str.maketrans(dict.fromkeys(punctuation))
setStopwords = set(stopwords.words('portuguese'))

# Realiza a tokenização e tratamento dos parágrafos do texto
def tokenize(corpus: str) -> list:
    corpusTokenized = []
    for paragraph in corpus:
        paragraph = paragraph.lower()                                       # Tratamento de case-sensitive
        paragraph = paragraph.encode('utf8', 'ignore').decode()             # Eliminação de caracteres fora de UTF-8
        paragraph = paragraph.translate(punctranslation)                    # Eliminação de pontuações
        corpusTokenized.append([token for token in paragraph.split()        # Eliminação de stopwords
                                if token not in setStopwords])
    return corpusTokenized
    
corpusTokenized = tokenize(corpus)

### 3. Contagem de unigramas do texto

In [123]:
unigrams = Counter()
for paragraph in corpusTokenized:
    for token in paragraph:
        unigrams[token] += 1

# Mapeamento de acesso (Token <-> Índice representativo)
token2index = {token: index for index, token in enumerate(unigrams.keys())}
index2token = {index: token for token, index in token2index.items()}

### 4. Contagem de bigramas do texto

In [124]:
# Contador de bigramas, considerando uma determinada janela de contexto (nesse caso, 2
# palavras antes e duas palavras depois)
skipgrams = Counter()
gap = 3

for paragraph in corpusTokenized:
    tokens = [token2index[tok] for tok in paragraph]
    
    # Para cada palavra no parágrafo, realiza a análise dos contextos da vizinhança
    for indexWord, word in enumerate(paragraph):
        indexContextMin = max(0, indexWord - gap)
        indexContextMax = min(len(paragraph)-1, indexWord + gap)

        # Para cada contexto da vizinhança, crie um bigrama com a palavra central
        indexContexts = [index for index in range(indexContextMin, indexContextMax + 1) if index != indexWord]
        for indexContext in indexContexts:
            skipgram = (tokens[indexWord], tokens[indexContext])
            skipgrams[skipgram] += 1

# Exibição dos 10 bigramas mais comuns presentes no texto, considerando a janela de contexto estipulada
mostCommon = [(index2token[skipgram[0][0]], index2token[skipgram[0][1]], skipgram[1]) 
               for skipgram in skipgrams.most_common(10)]
mostCommon

[('neve', 'neve', 12),
 ('alguma', 'coisa', 10),
 ('coisa', 'alguma', 10),
 ('idéia', 'fixa', 9),
 ('fixa', 'idéia', 9),
 ('mesma', 'coisa', 9),
 ('coisa', 'mesma', 9),
 ('sempre', 'mesma', 8),
 ('sempre', 'coisa', 8),
 ('mesma', 'sempre', 8)]

### 5. Criação da matrix de frequência termo-a-termo

In [125]:
# Mapeamento das entradas da matriz esparça de frequência entre os bigramas do texto
rowsMatrix = []
columnsMatrix = []
dataMatrix = []

for (token1, token2), skipgramCount in skipgrams.items():
    rowsMatrix.append(token1)
    columnsMatrix.append(token2)
    dataMatrix.append(skipgramCount)

wwMatrix = sparse.csr_matrix((dataMatrix, (rowsMatrix, columnsMatrix)))
wwMatrix

<3285x3285 sparse matrix of type '<class 'numpy.int64'>'
	with 52456 stored elements in Compressed Sparse Row format>

### 6. Criação da matriz PPMI

In [126]:
# Número total de bigramas presente na matriz de frequência
numSkipgrams = wwMatrix.sum()

# Mapeamento das entradas da matriz PPMI
rowsIndex = []
columnsIndex = []
ppmiData = []

# Vetor de frequência total de cada palavra em todos os possíveis contextos
sumWords = np.array(wwMatrix.sum(axis=0)).flatten()

# Vetor de frequência total de cada contexto para todas as possíveis palavras
sumContexts = np.array(wwMatrix.sum(axis=1)).flatten()

for (tokenWord, tokenContext), skipgramCount in skipgrams.items():

    # Frequência de determinada palavra em determinado contexto
    # [#(w,c)]
    freqWordContext = skipgramCount

    # Frequência de determinada palavra em todos os contextos possíveis
    # [#(w)]
    freqWord = sumContexts[tokenWord]
    
    # Frequência de determinado contexto para todas as palavras possíveis
    # [#(c)]
    freqContext = sumWords[tokenContext]

    # Probabilidade de ocorrência de determinada palavra em determinado contexto
    # [P(w,c)]
    probWordContext = freqWordContext / numSkipgrams

    # Probabilidade de ocorrência de determinada palavra individualmente
    # [P(w)]
    probWord = freqWord / numSkipgrams

    # Probabilidade de ocorrência de determinado contexto individualmente
    # [P(c)]
    probContext = freqContext / numSkipgrams

    # Cálculo PPMI (Positive Pointwise Mutual Information)
    # [PPMI = max(0, log( P(w,c)/(P(w)P(c)) ))]
    PPMI = max(np.log2(probWordContext / (probWord * probContext)), 0)

    rowsIndex.append(tokenWord)
    columnsIndex.append(tokenContext)
    ppmiData.append(PPMI)

ppmiMatrix = sparse.csr_matrix((ppmiData, (rowsIndex, columnsIndex)))
ppmiMatrix

<3285x3285 sparse matrix of type '<class 'numpy.float64'>'
	with 52456 stored elements in Compressed Sparse Row format>

### 7. Fatoração matricial usando SVD (Singular Value Decomposition)

In [127]:
from scipy.sparse.linalg import svds as SVD

# Dimensão proposta da matriz de valores singulares produzida pelo SVD
# [Hiperparâmetro]
embeddingSize = 50

U, D, V = SVD(ppmiMatrix, embeddingSize)

# Normalização das matrizes de vetores singulares produzidas pelo SVD
Unorm = U / np.sqrt(np.sum(U*U, axis=1, keepdims=True))
Vnorm = V / np.sqrt(np.sum(V*V, axis=1, keepdims=True))

# ???
wordVecs = Unorm

### 8. Visualização de palavras similares por similaridade por cosseno

In [128]:
from sklearn.metrics.pairwise import cosine_similarity

# Cálculo dos 10 contextos mais similares a dada palavra utilizando a matriz de Word Embedding
def wordsSimilarity(word, matrix, n):
    wordIndex = token2index[word]

    # Resgate do vetor representante de determinada palavra
    if isinstance(matrix, sparse.csr_matrix):
        wordVec = matrix.getrow(wordIndex)
    else:
        wordVec = matrix[wordIndex:wordIndex+1, :]

    # Cálculo da similidade (similaridade de vetores por cosseno)
    similarity = cosine_similarity(matrix, wordVec).flatten()
    sortedIndexes = np.argsort(-similarity)

    # Retorno dos n contextos mais similares a dada palavra
    similarityContextScores = [(index2token[sortedIndex], similarity[sortedIndex]) 
                                for sortedIndex in sortedIndexes[0:n]]

    return similarityContextScores

def wordSimilarityReport(word, matrix, n=10):
    similarityContextScores = wordsSimilarity(word, matrix, n)
    for context, similarity in similarityContextScores:
        print(f'Contexto: \'{context}\' \t Similaridade: {similarity}')

        wordParagraphs = [paragraph for paragraph in corpus 
                          if context in paragraph and word in paragraph][0:5]

        for paragraph in wordParagraphs:
            print(f'\"{paragraph}\"', end='\n\n')

wordSimilarityReport('defunto', wordVecs)

Contexto: 'defunto' 	 Similaridade: 1.0000000000000002
"Algum tempo hesitei se devia abrir estas memórias pelo princípio ou pelo fim, isto é, se poria em primeiro lugar o meu nascimento ou a minha morte. Suposto o uso vulgar seja começar pelo nascimento, duas considerações me levaram a adotar diferente método: a primeira é que eu não sou propriamente um autor defunto, mas um defunto autor, para quem a campa foi outro berço; a segunda é que o escrito ficaria assim mais galante e mais novo. Moisés, que também contou a sua morte, não a pôs no intróito, mas no cabo; diferença radical entre este livro e o Pentateuco."

"-Anda visitando os defuntos? Disse-lhe eu. -Ora, defuntos! respondeu Virgília com um muxoxo. E depois de me apertar as mãos: -Ando a ver se ponho os vadios para a rua."

"Logo depois, senti-me transformado na Suma Teologica de São Tomás, impressa num volume, e encadernada em marroquim, com fechos de prata e estampas; idéia esta que me deu ao corpo a mais completa imobilidade